# How to output KeplerLightCurve's to fits files

Once you have started detrending a lightcurve or altered a lightcurve in some way, you may want to save it as a fits file. This allows you to easily share the file with your collaborators or submit your lightcurves as a [MAST High Level Science Product](https://archive.stsci.edu/hlsp/hlsp_guidelines.html) (HLSP). `lightkurve` has the `to_fits()` method which will easily convert your `KeplerLightCurve` object into a fits file.

Below is a quick demostration of how `to_fits()` works.

*If you are considering contributing a HLSP you may want to read the [guidelines](https://archive.stsci.edu/hlsp/hlsp_guidelines_timeseries.html) for contributing fits files. These include which fits headers are required/suggested for your HLSP to be accepted.*

In [1]:
from lightkurve import KeplerLightCurveFile, KeplerTargetPixelFile

First we'll download a random lightcurve from MAST.

In [2]:
lcf = KeplerLightCurveFile.from_archive(757076, quarter=3, verbose=False)

Now we'll make some edits to the lightcurve, below we use the PDCSAP flux from MAST, remove NaN values and clip out any outliers.

In [ ]:
lc = lcf.PDCSAP_FLUX.remove_nans().remove_outliers()
lc.plot()

Now we can use the `to_fits` method to save the lightcurve to a file called *output.fits*.

In [ ]:
hdu = lc.to_fits(path='output.fits', overwrite=True)

Let's take a look at the file and check that it behaved as we expect

In [ ]:
from astropy.io import fits
hdu = fits.open('output.fits')
hdu

`hdu` is a set of astropy.io.fits objects, which is what we would expect. Lets take a look at the header of the first extension.

In [ ]:
hdu[0].header

Looks like it has all the correct information about the target. What about the second extension?

In [ ]:
hdu[1].header

This extension has 4 columns, `TIME`, `FLUX`, `FLUX_ERR` and `CADENCENO`. This is great! What about if we wanted to add new keywords to our fits file? HLSP products require some extra keywords. Let's add some keywords to explain who made the data, and what our HLSP is. 

In [ ]:
hdu = lc.to_fits(path='output.fits', overwrite=True,
                 hlsplead='Kepler/K2 GO office', hlspname='CLEANK2LC', citation='HEDGES2018')

In [ ]:
hdu[0].header

Now our new keywords are included in the primary header! What about if we want to add more data to our fits file? We can simply add this in the same way. Let's add the data quality to our fits file.

In [ ]:
hdu = lc.to_fits(path='output.fits', overwrite=True,
                 hlsplead='Kepler/K2 GO office', hlspname='CLEANK2LC', citation='HEDGES2018',
                 quality=lc.quality)

In [ ]:
hdu[1].header

Now the quality from our lightcurve appears in the second extension. Once all your lightcurves are saved as fits files and you have a README file, you can consider submitting your data produces to MAST.